# Workshop on sensor processing using machine learning
Course: Intelligent Sensing and Sense Making

Website: https://www.iss.nus.edu.sg/executive-education/course/detail/intelligent-sensing-and-sense--making/artificial-intelligence

Contact: Tian Jing

Email: tianjing@nus.edu.sg

## Objective
In this workshop, we will perform anomaly detection using autoencoder as reconstruction model

## Submission guideline
Once you finish the workshop, rename your .ipynb file to your name, and submit your .ipynb file into LumiNUS.


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
print("PyTorch version is", torch.__version__)
# Use GPU if available else revert to CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device being used:", device)


PyTorch version is 1.8.1+cu101
Device being used: cuda:0


In [ ]:
# Grant access to google drive.
# Run this cell, then you’ll see a link, click on that link, allow access
# Copy the code that pops up, paste it in the box, hit Enter

from google.colab import drive
drive.mount('/content/gdrive')

# Change working directory to be current folder
# os.chdir('/content/gdrive/My Drive/Your Folder Name/Your sub Folder Name')
os.chdir('/content/gdrive/My Drive/ISSM/machine')
!ls


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
archive  wk_abnormal_keras_v202107_reference.ipynb    wk_abnormal_v202107.ipynb
data	 wk_abnormal_pytorch_v202107_reference.ipynb


# Exercise: Machine health monitoring

- Reference: R. Zhao, R. Yan, Z. Chen, K. Mao, P. Wang, R. Gao, Deep learning and its applications to machine health monitoring, Mechanical Systems and Signal Processing, Vol. 115, Jan. 2019, pp. 213-237.

- pdf paper, https://arxiv.org/abs/1612.07640
- code, https://github.com/ClockworkBunny/MHMS_DEEPLEARNING

In [ ]:
# Load data 
x_train_pd = pd.read_csv("data/machine_sensor_train.csv",header=None)
x_train = x_train_pd.values
x_test_pd = pd.read_csv("data/machine_sensor_test.csv",header=None)
x_test = x_test_pd.values

print('Train shape: ', x_train.shape, 'Test shape:', x_test.shape)

Train shape:  (600, 70) Test shape: (300, 70)


In [ ]:

class myDataset(torch.utils.data.Dataset):
    def __init__(self, x_data):
        self.X = x_data
        self.num_samples = x_data.shape[0]

    def __getitem__(self, index):
        return self.X[index].astype('float32')

    def __len__(self):
        return self.num_samples

class myAEModel(nn.Module):
    def __init__(self, input_node, hidden_node):
        super(myAEModel, self).__init__()
        self.fc1 = nn.Sequential(nn.Linear(input_node, hidden_node), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(hidden_node, input_node)) # Note that the input signal has a range [-1,1], so we cannot use ReLU in the last layer

    def encode(self, x):
        z = self.fc1(x)
        return z  

    def decode(self, x):
        z = self.fc2(x)
        return z
        
    def forward(self, x):
        z = self.encode(x) 
        z = self.decode(z) 
        return z

In [ ]:
batch_size = 24
train_dataloader = torch.utils.data.DataLoader(myDataset(x_train), shuffle=True, batch_size=batch_size)
# # # Check dimension of a batch of training data
result = next(iter(train_dataloader))
print(result.shape)

input_node = x_train.shape[1]
hidden_node = 10
model = myAEModel(input_node, hidden_node)

model.to(device)
criterion = nn.MSELoss()
learning_rate = 1e-4
num_epoch = 20
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epoch):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        data = data.to(device)

        output = model(data)
        loss_mse = criterion(data, output)

        optimizer.zero_grad()
        loss_mse.backward()
        optimizer.step()

        running_loss += loss_mse.cpu().item()

    print("Epoch: %d, loss: %.4f" % (epoch+1, loss_mse.cpu().item()))

Epoch: 1, loss: 0.4588
Epoch: 2, loss: 0.4570
Epoch: 3, loss: 0.4427
Epoch: 4, loss: 0.3983
Epoch: 5, loss: 0.4217
Epoch: 6, loss: 0.4666
Epoch: 7, loss: 0.4266
Epoch: 8, loss: 0.3994
Epoch: 9, loss: 0.4028
Epoch: 10, loss: 0.3959
Epoch: 11, loss: 0.3654
Epoch: 12, loss: 0.3331
Epoch: 13, loss: 0.3451
Epoch: 14, loss: 0.3215
Epoch: 15, loss: 0.3058
Epoch: 16, loss: 0.3443
Epoch: 17, loss: 0.2854
Epoch: 18, loss: 0.2939
Epoch: 19, loss: 0.2926
Epoch: 20, loss: 0.2839


In [ ]:
sample_index = 0 # Select a test sample
thre = 10 # A manually-defined threshold

# Step 1: Select the test sample (such as the 0-th sample)
test_data_orig = x_test[sample_index:sample_index+1,:]
test_data_orig = torch.from_numpy(test_data_orig.astype('float32'))

model.eval()
with torch.no_grad():

        test_data_orig = test_data_orig.to(device)
        output = model(test_data_orig)
        dist = F.pairwise_distance(test_data_orig, output)
        print('Dissimlarity: %.4f' % (dist.item()))

if (dist>thre):
    print('Anomaly detected: the reconstruction error is %.2f' % dist)
else:
    print('Normal detected: the reconstruction error is %.2f' % dist)
    

Dissimlarity: 5.5625
Normal detected: the reconstruction error is 5.56


$\color{red}{\text{Q1. Complete the code to build autoencoder model for abnormal machine sensory signal detection.}}$

Tasks

- Peform the test on x_test and determine the test data (row index) with the LARGEST reconstruction error based on your trained model
- Plot the data that is found by your model

In [ ]:
# Provide your code to Q1 here
#
#


$\color{red}{\text{Q2. Describe one use case of anomaly detection, including objective, requirement of training data (if any).}}$

In [ ]:
# Provide your solution to Q2 here. (no need programming)
#
#

**Once you finish the workshop, rename your .ipynb file to your name, and submit your .ipynb file into LumiNUS.**

Have a nice day!